<a href="https://colab.research.google.com/github/Madushani-Weerasekara/D11_Bidirectional-LSTM_fake-news-classifier/blob/main/D11_Bidirectional_LSTM_fake_news_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fake News Classifier using Bidirectional LSTM RNN

In [298]:
import pandas as pd

In [299]:
df = pd.read_csv('/content/fake_or_real_news.csv', engine='python', error_bad_lines=False)

<ipython-input-299-af2d7ed5b0b4>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/fake_or_real_news.csv', engine='python', error_bad_lines=False)
Skipping line 1701: unexpected end of data


In [300]:
df.head()

Unnamed: 0                                              title  \
0        8476                       You Can Smell Hillary’s Fear   
1       10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2        3608        Kerry to go to Paris in gesture of sympathy   
3       10142  Bernie supporters on Twitter erupt in anger ag...   
4         875   The Battle of New York: Why This Primary Matters   

                                                text label  
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
2  U.S. Secretary of State John F. Kerry said Mon...  REAL  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE  
4  It's primary day in New York and front-runners...  REAL

In [301]:
df.shape

(1699, 4)

In [302]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [303]:
# Get Independent features
x = df.drop('label', axis=1)

In [304]:
# Dependent feature
y = df['label']

In [305]:
# Check wheather dataset is balance or not
y.value_counts()

FAKE    872
REAL    827
Name: label, dtype: int64

In [306]:
import tensorflow as tf

In [307]:
tf.__version__

'2.15.0'

In [308]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional

In [309]:
#Vocabulary size
voc_size = 5000

In [310]:
messages = x.copy()

In [311]:
messages.head()

Unnamed: 0                                              title  \
0        8476                       You Can Smell Hillary’s Fear   
1       10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2        3608        Kerry to go to Paris in gesture of sympathy   
3       10142  Bernie supporters on Twitter erupt in anger ag...   
4         875   The Battle of New York: Why This Primary Matters   

                                                text  
0  Daniel Greenfield, a Shillman Journalism Fello...  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  
2  U.S. Secretary of State John F. Kerry said Mon...  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  
4  It's primary day in New York and front-runners...

In [312]:
messages.shape

(1699, 3)

In [313]:
import nltk
import re
from nltk.corpus import stopwords

In [314]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [315]:
# If there are null values use this code fix missing index values
# messages.reset_index(inplace=True)

In [316]:
# Dataset preprocessing
from nltk.stem import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  print(i)
  review = re.sub('[^a-zA_Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [317]:
corpus

['ou mell illari ear',
 'atch xact oment aul yan ommit olit uicid rump alli',
 'erri go ari gestur sympathi',
 'erni support witter erupt anger e tri warn',
 'attl ew ork hy rimari atter',
 'ehran',
 'irl orrifi hat atch oyfriend e eft ace ime n',
 'ritain chindler i',
 'act check rump linton command chief forum',
 'ran reportedli make new push uranium concess nuclear talk',
 'ith three linton owa glimps fire elud illari linton campaign',
 'onald rump hockingli eak eleg ame omehow ot ven ors',
 'trong olar torm ech isk oday ew ct',
 'ay america repar orld ar',
 'rump take ruz lightli',
 'ow women lead differ',
 'hock ichel bama illari aught lamor ate ape romot',
 'illari linton roubl america otic hing idden ictur',
 'hat ran bill bama like',
 'chart explain everyth need know partisanship america',
 'slipperi slope rump propos ban uslim',
 'pisod ail eplor special guest andi',
 'illari linton ake ipartisan appeal taten sland',
 'ew enat major leader main goal scari',
 'nferno verpopul y

##Onehot Representation


In [318]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]
onehot_rep

[[2565, 1101, 4216, 2855],
 [2408, 1809, 2601, 2783, 4238, 4694, 1825, 2814, 3121, 4215],
 [23, 4229, 2652, 563, 835],
 [2825, 4527, 4493, 1580, 1916, 4534, 4395, 4387],
 [1923, 3114, 2403, 1986, 333, 1110],
 [4771],
 [4708, 3857, 477, 2408, 78, 4534, 3332, 616, 3719, 1325],
 [1666, 2764, 4904],
 [1331, 2524, 3121, 828, 999, 1794, 4983],
 [2543, 686, 2351, 3424, 2164, 756, 1029, 3953, 1254],
 [1527, 1386, 828, 2919, 2330, 4900, 3022, 4216, 828, 2376],
 [4730, 3121, 4244, 1509, 2707, 284, 3587, 2167, 4545, 4657],
 [2132, 3632, 3741, 1809, 1055, 2720, 3114, 1632],
 [4059, 2252, 615, 4559, 439],
 [3121, 4934, 3824, 1745],
 [61, 4332, 2435, 1808],
 [1488, 1783, 94, 4216, 1444, 3318, 4275, 3145, 4263],
 [4216, 828, 551, 2252, 4941, 3799, 1268, 2206],
 [477, 2543, 2751, 94, 2685],
 [2532, 1562, 4306, 2785, 1612, 2496, 2252],
 [1766, 3864, 3121, 2536, 4774, 3340],
 [3336, 3698, 1931, 3210, 999, 3681],
 [4216, 828, 3862, 969, 2595, 232, 3215],
 [3114, 4503, 174, 41, 4334, 1543, 518],
 [4895, 4

In [319]:
onehot_rep[0]

[2565, 1101, 4216, 2855]

##Embedding Representation

In [320]:
from sklearn.preprocessing import LabelEncoder

 # Initialize the label encoder
label_encoder = LabelEncoder()

# fit and transform on the training set
y_train_encoded = label_encoder.fit_transform(y_train)

#transform the test set
y_test_encoded = label_encoder.fit_transform(y_test)


In [321]:
#Check data types
print(y_train.dtype, y_test.dtype)

object object


In [322]:
#Print the unique classes in your target variable
print(np.unique(y_train))
print(np.unique(y_test))

['FAKE' 'REAL']
['FAKE' 'REAL']


In [323]:
#Check data types
print(y_train_encoded.dtype)
print(y_test_encoded.dtype)


int64
int64


In [324]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep, padding='post', maxlen=sent_length)
embedded_docs

array([[2565, 1101, 4216, ...,    0,    0,    0],
       [2408, 1809, 2601, ...,    0,    0,    0],
       [  23, 4229, 2652, ...,    0,    0,    0],
       ...,
       [1986, 2928, 4136, ...,    0,    0,    0],
       [3121, 1570, 4118, ...,    0,    0,    0],
       [1185, 1927, 3813, ...,    0,    0,    0]], dtype=int32)

In [325]:
# Creating Model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length)) #Add Embedding layer
model.add(Bidirectional(LSTM(100))) # Add BIdirectional LSTM with 100 neurons
model.add(Dense(1, activation='sigmoid')) # Add Dense layer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 20, 40)            200000    
                                                                 
 bidirectional_10 (Bidirect  (None, 200)               112800    
 ional)                                                          
                                                                 
 dense_10 (Dense)            (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [326]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [327]:
# Train test split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [328]:
model.fit(x_train, y_train_encoded, validation_data=(x_test, y_test_encoded), epochs=10, batch_size=32)


Epoch 1/10
36/36 [==============================] - 8s 133ms/step - loss: 0.6907 - accuracy: 0.5536 - val_loss: 0.6805 - val_accuracy: 0.6078
Epoch 2/10
36/36 [==============================] - 2s 46ms/step - loss: 0.4763 - accuracy: 0.8392 - val_loss: 0.4926 - val_accuracy: 0.7897
Epoch 3/10
36/36 [==============================] - 2s 45ms/step - loss: 0.1826 - accuracy: 0.9394 - val_loss: 0.5215 - val_accuracy: 0.7790
Epoch 4/10
36/36 [==============================] - 1s 19ms/step - loss: 0.0675 - accuracy: 0.9807 - val_loss: 0.6294 - val_accuracy: 0.7701
Epoch 5/10
36/36 [==============================] - 1s 25ms/step - loss: 0.0222 - accuracy: 0.9947 - val_loss: 1.0885 - val_accuracy: 0.7326
Epoch 6/10
36/36 [==============================] - 1s 20ms/step - loss: 0.0082 - accuracy: 0.9991 - val_loss: 1.1949 - val_accuracy: 0.7415
Epoch 7/10
36/36 [==============================] - 1s 17ms/step - loss: 0.0071 - accuracy: 0.9991 - val_loss: 1.2799 - val_accuracy: 0.7415
Epoch 8/10
3

In [334]:
y_pred = model.predict(x_test)
y_pred_classes = (y_pred > 0.5).astype('int32')  # Assuming binary classification with threshold 0.5

18/18 [==============================] - 0s 4ms/step


In [330]:
y_pred = np.where(y_pred>=0.5, 1,0)

In [335]:
y_pred_original = label_encoder.inverse_transform(y_pred_classes.flatten())

In [336]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred_original)

array([[228,  76],
       [ 71, 186]])

In [339]:
print(accuracy_score(y_test, y_pred_original))

0.7379679144385026
